# Supply Chain Network Optimization (Simulated Case)

**Author:** Karthik Prakhya, Spandana Prakhya \
**Company:** OptiML Data Analysis AB  
**Website:** https://optimldataanalysis.se  
**Contact:** info@optimldataanalysis.se   

**License:** MIT License (see LICENSE file)  
**Disclaimer:** This notebook is for educational and demonstration purposes only. All data used are synthetic or publicly available; no confidential or client-specific information is included.

---


## Executive Summary

This notebook is a demonstration of methods used in supply chain network optimization and multi-echelon inventory planning. It uses synthetic or public datasets and is intended to illustrate methodology, not to present results from client engagements.

**Key objectives:**
- Demonstrate approach and pipeline
- Provide a reproducible example for technical reviewers
- Highlight business impact where appropriate

**Tools Used:** Python, NumPy, pandas, matplotlib, and standard optimization/ML libraries (e.g., OR-Tools, Gurobi, scikit-learn, TensorFlow/PyTorch).

---


# MILP Supply-Chain Examples — Comprehensive Notebook

This notebook brings together:

- Basic MILP models: Greenfield facility location, Brownfield + CAPEX, Sourcing with MOQs.
- Extensions: Multi-commodity flows, Inventory with lead time & service level requirements.
- Benders decomposition implemented using Gurobi callbacks (master + LP subproblem loop).
- Mock AWS integration examples (mock S3, Parameter Store, Batch) and real `boto3` snippets.

Each model has a **mathematical explanation** (LaTeX) followed by a **code cell** (Pyomo). The Benders section demonstrates an iterative Pyomo implementation that uses Gurobi as the solver for both master and subproblem. Replace mock AWS classes with real `boto3` calls in production.  

**Note:** This notebook contains runnable code cells. It expects `pyomo` and a solver (Gurobi recommended). If Gurobi is not available, change solver names to `glpk`/`cbc` where appropriate.


## 1) Greenfield Analysis — Math & Explanation

The goal of the following Greenfield analysis to decide which facilities to keep open to serve and which facility will serve which customer.The following MILP model extends the Greenfield scenario by minimizing the total cost of operation, which comprises:  

1. **Transportation cost** of shipping products from open facilities to customers,  
2. **Operating cost** of running facilities,  
3. **Demand** for each customer, and 
4. **Production Capacity** of each facility.

**Sets & Indices**:
- $I$: candidate facility indices (plants/warehouses).
- $J$: customer indices.

**Parameters**:
- $d_j$: demand for customer $j$.
- $f_i$: fixed opening cost for facility $i$.
- $c_{ij}$: per-unit transportation cost from facility $i$ to customer $j$.
- $u_i$: capacity of facility $i$.

**Decision variables**:
- $y_i \in \{0,1\}$: 1 if facility $i$ is opened.
- $x_{ij} \ge 0$: flow from facility $i$ to customer $j$.

**Model:**

$$\min \sum_{i\in I} f_i y_i + \sum_{i\in I}\sum_{j\in J} c_{ij} x_{ij}$$
subject to
$$\sum_{i\in I} x_{ij} = d_j \quad\forall j\in J$$
$$\sum_{j\in J} x_{ij} \le u_i y_i \quad\forall i\in I$$


In [2]:
# Greenfield facility location — Pyomo implementation
from pyomo.environ import *

def greenfield_pyomo(solver_name='gurobi'):
    I = ['F1','F2','F3']
    J = ['C1','C2','C3','C4']
    d = {'C1':80,'C2':70,'C3':60,'C4':40}
    f = {'F1':10000,'F2':12000,'F3':9000}
    u = {'F1':200,'F2':150,'F3':120}
    c = {}
    for i in I:
        for j in J:
            c[(i,j)] = 2 + (0 if i=='F1' else (1 if i=='F2' else 3)) + (0 if j=='C1' else 1)

    model = ConcreteModel()
    model.I = Set(initialize=I)
    model.J = Set(initialize=J)
    model.d = Param(model.J, initialize=d)
    model.f = Param(model.I, initialize=f)
    model.u = Param(model.I, initialize=u)
    model.c = Param(model.I, model.J, initialize=lambda m,i,j: c[(i,j)])

    model.y = Var(model.I, domain=Binary)
    model.x = Var(model.I, model.J, domain=NonNegativeReals)

    def obj_rule(m):
        return sum(m.f[i]*m.y[i] for i in m.I) + sum(m.c[i,j]*m.x[i,j] for i in m.I for j in m.J)
    model.OBJ = Objective(rule=obj_rule, sense=minimize)

    def demand_rule(m,j):
        return sum(m.x[i,j] for i in m.I) == m.d[j]
    model.Demand = Constraint(model.J, rule=demand_rule)

    def cap_rule(m,i):
        return sum(m.x[i,j] for j in m.J) <= m.u[i]*m.y[i]
    model.Capacity = Constraint(model.I, rule=cap_rule)

    solver = SolverFactory(solver_name)
    print("Solving Greenfield with solver:", solver_name)
    results = solver.solve(model, tee=True)
    print('Objective:', value(model.OBJ))
    for i in I:
        print(i, 'open=', int(round(value(model.y[i]))))
    for i in I:
        for j in J:
            q = value(model.x[i,j])
            if q > 1e-6:
                print('Ship', q, 'from', i, 'to', j)

# Run the Greenfield analysis example
greenfield_pyomo('gurobi')


Solving Greenfield with solver: gurobi
Read LP format model from file /tmp/tmpzfs50l0o.pyomo.lp
Reading time = 0.00 seconds
x1: 7 rows, 15 columns, 27 nonzeros
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 24.04.1 LTS")

CPU model: 12th Gen Intel(R) Core(TM) i7-12700H, instruction set [SSE2|AVX|AVX2]
Thread count: 20 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 7 rows, 15 columns and 27 nonzeros
Model fingerprint: 0x37b017e9
Variable types: 12 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [2e+00, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+01, 8e+01]
Presolve time: 0.00s
Presolved: 7 rows, 15 columns, 27 nonzeros
Variable types: 12 continuous, 3 integer (3 binary)
Found heuristic solution: objective 31720.000000

Root relaxation: objective 1.457000e+04, 11 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Obj

The above solution says that only facilities F1 and F3 are kept open, 80 units are shipped from F1 to C1, 20 units are shipped from F1 to C2, 60 units are shipped F1 to C3, 40 units are shipped F1 to C4 and 50 units are shipped from F3 to C2. The total cost of operation is 19820.

## 2) Brownfield + CAPEX — Math & Explanation

Brownfield + CAPEX analysis is used to plan expansion or optimization of an existing supply-chain network.  In this simple example, we consider existing capacities at various facility sites and candidate expansion chunks at each site, and we decide whether to apply a certain expansion to a given site or not based on the CAPEX cost of that expansion and how much additional production capacity that expansion adds to the network.  

The following MILP model extends the Greenfield scenario by minimizing the total cost of operation, which comprises:  
1. **Transportation cost** of shipping products from open facilities to customers,  
2. **Operating cost** of running facilities per unit,
3. **Demand** for each customer, and 
4. **CAPEX cost** of all planned expansions (plus optional closure penalties), and 
5. **Production Capacity** of each facility, and
6. **Potential added capacity** at each facility given a certain expansion.

**Sets & Indices**  
- $I$: existing facility sites  
- $J$: customers  
- $K$: expansion chunk indices at each site  

**Parameters**  
- $u_i^0$: existing capacity at site $i$  
- $\Delta u_{i,k}$: capacity added by chunk $ k $ at site $ i $  
- $g_{i,k}$: CAPEX cost for chunk $ k $ at site $ i $  
- $h_i$: closure cost or fixed operating penalty  
- $o_i$: per-unit operating cost for site $ i $  
- $c_{ij}$: transport cost from $ i $ to $ j $  
- $d_j$: demand of customer $ j $  

**Decision Variables**  
- $y_i \in \{0,1\}$: 1 if site $i$ is kept open  
- $z_{i,k} \in [0,1]$: proportion of expansion chunk $k$ applied at site $i$  
- $ x_{ij} \geq 0 $: shipment quantity from site $ i $ to customer $ j $  

**Model**

$$
\min \quad 
\sum_{i,k} g_{i,k} z_{i,k}
 + \sum_{i,j} c_{ij} x_{ij}
 + \sum_i h_i (1 - y_i)
 + \sum_i o_i \sum_j x_{ij} $$
subject to
$$\sum_i x_{ij} = d_j, \forall j $$
$$\sum_j x_{ij} \leq u_i^0 y_i + \sum_k \Delta u_{i,k} z_{i,k}, \forall i$$
$$z_{i,k} \leq y_i, \forall i,k$$

In [18]:
# Brownfield + CAPEX example (Pyomo)
from pyomo.environ import *

def run_brownfield_pyomo(solver_name='gurobi'):
    # ----- Sets -----
    sites = ['S1', 'S2', 'S3']
    customers = ['C1', 'C2', 'C3']

    # ----- Parameters -----
    demand = {'C1': 120, 'C2': 100, 'C3': 80}
    cap0 = {'S1': 100, 'S2': 80, 'S3': 60}

    # Expansion chunks (capacity & CAPEX)
    delta = {
        ('S1',1): 100, ('S1',2): 100,
        ('S2',1): 150, ('S2',2): 150,
        ('S3',1): 80,  ('S3',2): 80
    }
    g = {
        ('S1',1): 200, ('S1',2): 200,
        ('S2',1): 300, ('S2',2): 300,
        ('S3',1): 150, ('S3',2): 150
    }

    # Transport cost ($/unit)
    c = {
        ('S1','C1'): 5, ('S1','C2'): 6, ('S1','C3'): 8,
        ('S2','C1'): 7, ('S2','C2'): 5, ('S2','C3'): 6,
        ('S3','C1'): 8, ('S3','C2'): 7, ('S3','C3'): 4
    }

    # Fixed closure/operating penalty and variable operating cost
    close_cost = {'S1': 4000, 'S2': 3500, 'S3': 3000}
    oper_cost  = {'S1': 1.0,  'S2': 1.2,  'S3': 0.9}

    # ----- Model -----
    model = ConcreteModel()
    model.I = Set(initialize=sites)
    model.J = Set(initialize=customers)
    model.K = Set(initialize=[1,2])

    model.d = Param(model.J, initialize=demand)
    model.cap0 = Param(model.I, initialize=cap0)
    model.delta = Param(model.I, model.K, initialize=lambda m,i,k: delta.get((i,k),0))
    model.g = Param(model.I, model.K, initialize=lambda m,i,k: g.get((i,k),0))
    model.c = Param(model.I, model.J, initialize=lambda m,i,j: c[(i,j)])
    model.h = Param(model.I, initialize=close_cost)
    model.op = Param(model.I, initialize=oper_cost)

    # ----- Decision Variables -----
    model.y = Var(model.I, domain=Binary)          # keep site
    model.z = Var(model.I, model.K, domain=UnitInterval)  # partial expansion allowed
    model.x = Var(model.I, model.J, domain=NonNegativeReals)

    # ----- Objective -----
    def obj_rule(m):
        return (
            sum(m.g[i,k]*m.z[i,k] for i in m.I for k in m.K) +
            sum(m.c[i,j]*m.x[i,j] for i in m.I for j in m.J) +
            sum(m.h[i]*(1 - m.y[i]) for i in m.I) +
            sum(m.op[i]*sum(m.x[i,j] for j in m.J) for i in m.I)
        )
    model.OBJ = Objective(rule=obj_rule, sense=minimize)

    # ----- Constraints -----
    def demand_rule(m, j):
        return sum(m.x[i,j] for i in m.I) == m.d[j]
    model.Demand = Constraint(model.J, rule=demand_rule)

    def cap_rule(m, i):
        return sum(m.x[i,j] for j in m.J) <= m.cap0[i]*m.y[i] + sum(m.delta[i,k]*m.z[i,k] for k in m.K)
    model.Capacity = Constraint(model.I, rule=cap_rule)

    def expand_if_open(m, i, k):
        return m.z[i,k] <= m.y[i]
    model.ExpandIfOpen = Constraint(model.I, model.K, rule=expand_if_open)

    # ----- Solve -----
    solver = SolverFactory(solver_name)
    solver.solve(model, tee=True)

    # ----- Results -----
    print("\nOptimal Objective:", value(model.OBJ))
    for i in sites:
        print(f"{i}: open={int(round(value(model.y[i])))}")
        for k in [1,2]:
            if value(model.z[i,k]) > 1e-4:
                print(f"  expansion chunk {k}: {value(model.z[i,k]):.2f}")
    for i in sites:
        for j in customers:
            q = value(model.x[i,j])
            if q > 1e-6:
                print(f"Ship {q:.1f} units from {i} → {j}")

# Run the example
run_brownfield_pyomo('gurobi')

Read LP format model from file /tmp/tmptvibf_hf.pyomo.lp
Reading time = 0.00 seconds
x1: 12 rows, 19 columns, 39 nonzeros
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 24.04.1 LTS")

CPU model: 12th Gen Intel(R) Core(TM) i7-12700H, instruction set [SSE2|AVX|AVX2]
Thread count: 20 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 12 rows, 19 columns and 39 nonzeros
Model fingerprint: 0x67fb68bb
Variable types: 16 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [5e+00, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e+01, 1e+02]
Presolve removed 6 rows and 7 columns
Presolve time: 0.00s
Presolved: 6 rows, 12 columns, 21 nonzeros
Variable types: 12 continuous, 0 integer (0 binary)

Root relaxation: objective 1.849500e+03, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj

The above solution says that all three sites S1, S2 and S3 should be open. As far as expansion goes, 20 % of expansion chunk 1 should be applied to site S1, 13 % of expansion chunk 2 should be applied to site S2 and 25 % of expansion chunk 1 should be applied to site S3. Finally, 120 units should be shipped from site S1 to customer C1, 100 units should be shipped from site S2 to customer C2 and 80 units should be shipped from site S3 to customer C3.

## 3) Multi-period Brownfield Expansion (Staged CAPEX)

When expansions are planned over multiple periods (years), we must model:
- Periodic demand and shipments,
- CAPEX timing and discounting,
- Cumulative capacity increases from expansions purchased in earlier periods,
- Operating and transport costs each period.

**Sets & Indices**  
- $I$: existing facility sites.  
- $J$: customers.  
- $K$: expansion chunk indices at each site.  
- $T$: planning periods (years), $t=1,\dots,T$.

**Parameters**  
- $d_{j,t}$: demand of customer $j$ in period $t$.  
- $u_i^0$: initial capacity at site $i$ (available from $t=1$).  
- $\Delta u_{i,k}$: capacity added by chunk $k$ at site $i$ (one-time capacity addition).  
- $g_{i,k}$: CAPEX cost for chunk $k$ at site $i$ (incurred in the period when purchased).  
- $c_{ij,t}$: transport cost from $i$ to $j$ in period $t$.  
- $h_{i,t}$: operating/fixed cost for keeping site $i$ open in period $t$ (could capture staffing, overhead).  
- $o_{i,t}$: per-unit operating cost at site $i$ in period $t$ (variable cost on shipped units).  
- $\gamma$: discount factor per period (e.g. $\gamma = 1/(1+r)$ for interest rate $r$).  
- Optional: $y_{i,0}$ (initial open status).

**Decision variables**  
- $y_{i,t}\in\{0,1\}$: 1 if site $i$ is operated in period $t$.  
- $z_{i,k,t}\in\{0,1\}$: 1 if expansion chunk $k$ at $i$ is purchased in period $t$ (one-time purchase).  
- $x_{i,j,t}\ge 0$: flow shipped from $i$ to $j$ in period $t$.

**Key modeling choices**  
- Capacity is cumulative: capacity available in period $t$ equals $u_i^0$ plus all $\Delta u_{i,k}$ purchased in periods $\tau \le t$.  
- If required, force monotonic $y$ (once opened remains open) with $y_{i,t} \ge y_{i,t-1}$; or allow closing by not imposing monotonicity.  
- CAPEX incurred in the period of purchase and discounted back if desired in the objective.

**Model (periodic objective with discounting)**

$$
\min \quad
\sum_{t=1}^T \gamma^{\,t-1}\Bigg(
    \sum_{i,k} g_{i,k}\, z_{i,k,t}
  + \sum_{i,j} c_{ij,t}\, x_{i,j,t}
  + \sum_i h_{i,t}\,(1 - y_{i,t})
  + \sum_i o_{i,t}\sum_j x_{i,j,t}
\Bigg)
$$

subject to demand satisfaction (each period):

$$
\sum_{i \in I} x_{i,j,t} = d_{j,t} \quad\forall j,\; t=1,\dots,T
$$

cumulative capacity constraint (period $t$):

$$
\sum_{j \in J} x_{i,j,t} \le u_i^0\,y_{i,t} + \sum_{\tau=1}^t \sum_{k\in K} \Delta u_{i,k}\, z_{i,k,\tau}
\quad\forall i,\; t=1,\dots,T
$$

expansions only if site is open that period (purchase requires the site being active that same period):

$$
z_{i,k,t} \le y_{i,t} \quad\forall i,k,t
$$

(option) force non-decreasing open status:

$$
y_{i,t} \ge y_{i,t-1} \quad\forall i,\; t\ge 2
$$

This multi-period structure allows early investments to benefit later periods and lets the solver trade off early CAPEX vs. later operating and transport costs.

In [22]:
# Multi-period Brownfield + CAPEX (Pyomo)
from pyomo.environ import *

def run_multiperiod_brownfield(solver_name='gurobi'):
    # ----- Sets -----
    sites = ['S1','S2','S3']
    customers = ['C1','C2','C3']
    periods = [1,2,3]   # 3-year planning horizon

    # ----- Parameters -----
    d = { ( 'C1',1):120, ('C2',1):100, ('C3',1):80,
          ( 'C1',2):130, ('C2',2):110, ('C3',2):85,
          ( 'C1',3):140, ('C2',3):120, ('C3',3):90 }
    cap0 = {'S1':100, 'S2':80, 'S3':60}
    delta = {('S1',1):100, ('S1',2):100,
             ('S2',1):150, ('S2',2):150,
             ('S3',1):80,  ('S3',2):80}
    g = {('S1',1):200, ('S1',2):200,
         ('S2',1):300, ('S2',2):300,
         ('S3',1):150, ('S3',2):150}
    # transport can vary by period (small inflation or changing lanes)
    c = {('S1','C1',1):5, ('S1','C2',1):6, ('S1','C3',1):8,
         ('S2','C1',1):7, ('S2','C2',1):5, ('S2','C3',1):6,
         ('S3','C1',1):8, ('S3','C2',1):7, ('S3','C3',1):4}

    # simple approach: reassign for all t
    c = {}
    for t in periods:
        infl = 1.0 + 0.02*(t-1)   # 2% per year
        c.update({
            ('S1','C1',t):5*infl, ('S1','C2',t):6*infl, ('S1','C3',t):8*infl,
            ('S2','C1',t):7*infl, ('S2','C2',t):5*infl, ('S2','C3',t):6*infl,
            ('S3','C1',t):8*infl, ('S3','C2',t):7*infl, ('S3','C3',t):4*infl,
        })

    h = { ( 'S1',1):4000, ('S2',1):3500, ('S3',1):3000,
          ( 'S1',2):4000, ('S2',2):3500, ('S3',2):3000,
          ( 'S1',3):4000, ('S2',3):3500, ('S3',3):3000 }
    op = { ( 'S1',1):1.0, ('S2',1):1.2, ('S3',1):0.9,
           ( 'S1',2):1.0, ('S2',2):1.2, ('S3',2):0.9,
           ( 'S1',3):1.0, ('S2',3):1.2, ('S3',3):0.9 }

    gamma = 1.0 / 1.05   # discount per year (r=5%)

    # ----- Model -----
    model = ConcreteModel()
    model.I = Set(initialize=sites)
    model.J = Set(initialize=customers)
    model.K = Set(initialize=[1,2])
    model.T = Set(initialize=periods)

    model.d = Param(model.J, model.T, initialize=lambda m,j,t: d[(j,t)])
    model.cap0 = Param(model.I, initialize=cap0)
    model.delta = Param(model.I, model.K, initialize=lambda m,i,k: delta.get((i,k),0))
    model.g = Param(model.I, model.K, initialize=lambda m,i,k: g.get((i,k),0))
    model.c = Param(model.I, model.J, model.T, initialize=lambda m,i,j,t: c[(i,j,t)])
    model.h = Param(model.I, model.T, initialize=lambda m,i,t: h[(i,t)])
    model.op = Param(model.I, model.T, initialize=lambda m,i,t: op[(i,t)])

    # Decision vars
    model.y = Var(model.I, model.T, domain=Binary)
    model.z = Var(model.I, model.K, model.T, domain=Binary)
    model.x = Var(model.I, model.J, model.T, domain=NonNegativeReals)

    # Objective (discounted)
    def obj_rule(m):
        return sum((gamma**(t-1)) * (
            sum(m.g[i,k]*m.z[i,k,t] for i in m.I for k in m.K)
            + sum(m.c[i,j,t]*m.x[i,j,t] for i in m.I for j in m.J)
            + sum(m.h[i,t]*(1 - m.y[i,t]) for i in m.I)
            + sum(m.op[i,t]*sum(m.x[i,j,t] for j in m.J) for i in m.I)
        ) for t in m.T)
    model.OBJ = Objective(rule=obj_rule, sense=minimize)

    # Demand
    def demand_rule(m,j,t):
        return sum(m.x[i,j,t] for i in m.I) == m.d[j,t]
    model.Demand = Constraint(model.J, model.T, rule=demand_rule)

    # Capacity: cumulative expansions purchased up to period t
    def cap_rule(m,i,t):
        return sum(m.x[i,j,t] for j in m.J) <= m.cap0[i]*m.y[i,t] + \
            sum(m.delta[i,k]*sum(m.z[i,k,tt] for tt in m.T if tt <= t) for k in m.K)
    model.Capacity = Constraint(model.I, model.T, rule=cap_rule)

    # expansions only when open in that period
    def expand_if_open(m,i,k,t):
        return m.z[i,k,t] <= m.y[i,t]
    model.ExpandIfOpen = Constraint(model.I, model.K, model.T, rule=expand_if_open)

    # optional: once opened, remain open (uncomment if desired)
    # def nondec_open(m,i,t):
    #     if t == min(m.T): return Constraint.Skip
    #     prev = sorted(list(m.T))[sorted(list(m.T)).index(t)-1]
    #     return m.y[i,t] >= m.y[i,prev]
    # model.NonDecOpen = Constraint(model.I, model.T, rule=nondec_open)

    # Solve
    solver = SolverFactory(solver_name)
    solver.solve(model, tee=True)

    # Results (simple display)
    print("\nOptimal Objective:", value(model.OBJ))
    for t in periods:
        print(f"\n--- Period {t} ---")
        for i in sites:
            print(f"{i}: open={int(round(value(model.y[i,t])))}")
            for k in [1,2]:
                if value(model.z[i,k,t]) > 1e-6:
                    print(f"  purchased chunk {k} in period {t}")
        for i in sites:
            for j in customers:
                q = value(model.x[i,j,t])
                if q > 1e-6:
                    print(f"  Ship {q:.1f} from {i} -> {j}")

run_multiperiod_brownfield('gurobi')

Read LP format model from file /tmp/tmp9dfzdfgl.pyomo.lp
Reading time = 0.00 seconds
x1: 36 rows, 55 columns, 135 nonzeros
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 24.04.1 LTS")

CPU model: 12th Gen Intel(R) Core(TM) i7-12700H, instruction set [SSE2|AVX|AVX2]
Thread count: 20 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 36 rows, 55 columns and 135 nonzeros
Model fingerprint: 0xc5213bc1
Variable types: 28 continuous, 27 integer (27 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [5e+00, 3e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e+01, 1e+02]
Found heuristic solution: objective 7768.5895692
Presolve removed 18 rows and 19 columns
Presolve time: 0.00s
Presolved: 18 rows, 36 columns, 72 nonzeros
Variable types: 27 continuous, 9 integer (0 binary)

Root relaxation: objective 5.656651e+03, 11 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |

As the output shows, the solution to the above MILP provides the following information for each of the three periods: (1) Whether the sites S1, S2 and S3 are open or not, (2) whether an expansion chunk was purchased for any of the three sites and (3) how many units of the product were shipped from each of the three sites to the three customers. From the solution, it is clear that the customer-site mapping was not changed from period-to-period, expansion chunk 1 and 2 were purchased in period 1 for sites S1 and S3, respectively and the units shipped changed from period to period.

## 3) Sourcing with MOQs — Math & Explanation

It is also possible to handle constraints such as minimum order quanitities (MOQs) when handing sourcing of materials for a supply chain. The following MILP model incorporates **tiered per-unit pricing** and **rebates** to reflect realistic supplier contracts.  
Each supplier $s$ can offer:

1. **Tiered pricing**: cost per unit decreases when quantity exceeds predefined breakpoints.  
2. **Rebates**: a fixed reduction per unit if total quantity exceeds a threshold.  

**Sets and Indices**

- $s \in S$: suppliers  
- $t \in T_s$: pricing tiers for supplier $s$  


**Parameters**

- $D$: total demand  
- $K_s$: maximum capacity for supplier $s$  
- $F_s$: fixed activation cost for supplier $s$  
- $m_s$: minimum order quantity (optional)  
- $p_{s,t}$: per-unit price at tier $t$ of supplier $s$  
- $Q_{s,t}$: upper limit of tier $t$ for supplier $s$  
- $R_s$: rebate per unit if $q_s \ge Q_s^{rebate}$  

**Decision Variables**

- $y_s \in \{0,1\}$: supplier activation  
- $q_s \ge 0$: total quantity from supplier $s$  
- $q_{s,t} \ge 0$: quantity from supplier $s$ in tier $t$  
- $b_s \in \{0,1\}$: rebate indicator (1 if $q_s \ge Q_s^{rebate}$)  

**Objective Function**

$$
\min \sum_s F_s y_s + \sum_s \sum_t p_{s,t} q_{s,t} - \sum_s R_s b_s q_s
$$

- First term: fixed supplier costs  
- Second term: tiered per-unit cost  
- Third term: rebate applied if threshold exceeded  

**Demand satisfaction Constraint**:

$$
\sum_s q_s = D, \quad q_s = \sum_t q_{s,t} \quad \forall s
$$

**Tier Capacity Constraints**:

$$
0 \le q_{s,t} \le Q_{s,t} - Q_{s,t-1}, \quad \forall s,t
$$

**Supplier capacity Constraint**:

$$
q_s \le K_s y_s \quad \forall s
$$

**Minimum order (optional)**:

$$
q_s \ge m_s y_s \quad \forall s
$$

**Rebate logic (threshold)**:

$$
q_s - Q_s^{rebate} \le M_s b_s, \quad q_s - Q_s^{rebate} \ge \epsilon b_s
$$

- $M_s$ is a big-M parameter  
- $\epsilon$ is a small positive constant  

In [33]:
from pyomo.environ import *


def run_sourcing_brown(solver_name='gurobi'):
    # ----- Feasible sample data -----
    suppliers = ['S1','S2']
    tiers = { 'S1':[1,2], 'S2':[1,2] }

    D = 150                   # total demand
    F = {'S1': 500, 'S2': 300}       # fixed costs
    m = {'S1': 20, 'S2': 30}         # minimum orders
    K = {'S1': 100, 'S2': 120}       # max capacities

    # Tiered pricing and tier upper limits
    p = {('S1',1):10, ('S1',2):8,
         ('S2',1):12, ('S2',2):10}
    Q = {('S1',1):50, ('S1',2):100,
         ('S2',1):60, ('S2',2):120}

    # Rebates
    R = {'S1':1, 'S2':2}
    Q_rebate = {'S1':80, 'S2':90}

    # ----- Pyomo Model -----
    model = ConcreteModel()
    model.S = Set(initialize=suppliers)
    model.T = Set(model.S, initialize=lambda m,s: tiers[s])

    # Decision variables
    model.y = Var(model.S, domain=Binary)
    model.q = Var(model.S, domain=NonNegativeReals)
    model.qt = Var(((s,t) for s in suppliers for t in tiers[s]), domain=NonNegativeReals)
    model.b = Var(model.S, domain=Binary)

    # Parameters as Pyomo Params
    model.m_param = Param(model.S, initialize=m)
    model.K_param = Param(model.S, initialize=K)
    model.Q_rebate = Param(model.S, initialize=Q_rebate)
    model.R_param = Param(model.S, initialize=R)
    model.F_param = Param(model.S, initialize=F)

    # Objective: fixed + tiered cost - rebate
    def obj_rule(m):
        tier_cost = sum(p[s,t]*m.qt[s,t] for s in m.S for t in m.T[s])
        rebate = sum(m.R_param[s]*m.b[s]*m.q[s] for s in m.S)
        fixed = sum(m.F_param[s]*m.y[s] for s in m.S)
        return fixed + tier_cost - rebate
    model.OBJ = Objective(rule=obj_rule, sense=minimize)

    # Demand satisfaction
    model.Demand = Constraint(expr=sum(model.q[s] for s in model.S) == D)

    # Link total quantity to tiered quantities
    def tier_link_rule(m,s):
        return m.q[s] == sum(m.qt[s,t] for t in m.T[s])
    model.TierLink = Constraint(model.S, rule=tier_link_rule)

    # Tier limits
    def tier_limit_rule(m,s,t):
        lower = Q.get((s,t-1),0) if t>1 else 0
        return m.qt[s,t] <= Q[s,t]-lower
    model.TierLimit = Constraint(((s,t) for s in suppliers for t in tiers[s]), rule=tier_limit_rule)

    # Capacity constraint
    def cap_rule(m,s):
        return m.q[s] <= m.K_param[s]*m.y[s]
    model.Capacity = Constraint(model.S, rule=cap_rule)

    # Minimum order
    def min_rule(m,s):
        return m.q[s] >= m.m_param[s]*m.y[s]
    model.MinOrder = Constraint(model.S, rule=min_rule)

    # Rebate constraints with corrected Big-M logic
    M = 1e5
    epsilon = 1e-3
    def rebate_upper(m,s):
        return m.q[s] <= m.Q_rebate[s] - epsilon + M*m.b[s]
    def rebate_lower(m,s):
        return m.q[s] >= m.Q_rebate[s]*m.b[s]
    model.RebateUpper = Constraint(model.S, rule=rebate_upper)
    model.RebateLower = Constraint(model.S, rule=rebate_lower)

    # Solve
    solver = SolverFactory('gurobi')
    solver.solve(model, tee=True)

    # Results
    for s in suppliers:
        print(f"{s}: activated={int(round(model.y[s].value))}, total={model.q[s].value:.1f}, rebate={int(round(model.b[s].value))}")
        for t in tiers[s]:
            if model.qt[s,t].value>0:
                print(f"  tier {t}: {model.qt[s,t].value:.1f} units")


run_sourcing_brown('gurobi')

Read LP format model from file /tmp/tmp4koqkw5r.pyomo.lp
Reading time = 0.00 seconds
x1: 15 rows, 10 columns, 28 nonzeros
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 24.04.1 LTS")

CPU model: 12th Gen Intel(R) Core(TM) i7-12700H, instruction set [SSE2|AVX|AVX2]
Thread count: 20 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 15 rows, 10 columns and 28 nonzeros
Model fingerprint: 0x1b151cde
Model has 2 quadratic objective terms
Variable types: 6 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+05]
  Objective range  [8e+00, 5e+02]
  QObjective range [2e+00, 4e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e+01, 2e+02]
Presolve removed 8 rows and 2 columns
Presolve time: 0.00s
Presolved: 9 rows, 10 columns, 24 nonzeros
Variable types: 8 continuous, 2 integer (2 binary)
Found heuristic solution: objective 2220.0020000

Root relaxation: objective 2.032000e+03, 7 iterations, 0.00 

The optimal solution to the sourcing MILP says that 60 units were sourced from supplier S1 with 10 units at tier 1 pricing, 50 units at tier 2 pricing with no rebate and 90 units were sourced from supplier S2 with 30 units at tier 1 pricing and 60 units at tier 2 pricing with a rebate applied. It is possible to similarly extend this model to multi-period and to integrate this with the MILP for Brownfield + CAPEX analysis discussed earlier to create a unified supply chain model for end-to-end optimization. 

## 9) Multi-Commodity, Multi-Product Supply Chain Model with BOM Conversion

This model extends the two-echelon supply chain to support **multiple raw materials** and **multiple products**, linked through plant-level **bill-of-materials (BOM)** conversion factors.

**Sets and Indices**

- $S$: suppliers  
- $I$: plants  
- $J$: customers  
- $R$: raw materials  
- $P$: finished products  

**Parameters**
- $d_{j,p}$: demand of product $p$ at customer $j$ 
- $\text{Cap}^R_{s,r}$: capacity of supplier $s$ for raw material $r$
- $\text{Cap}^P_i$: production capacity of plant $i$
- $F_i$: fixed (opening) cost of plant $i$ 
- $c^R_{s,i,r}$: unit transport cost of raw material $r$ from supplier $s$ to plant $i$
- $c^P_{i,j,p}$: unit transport cost of product $p$ from plant $i$ to customer $j$ 
- $\alpha_{i,r,p}$: BOM: units of raw material $r$ required at plant $i$ per unit of product $p$ produced

---

**Decision Variables**
- $y_i \in \{0,1\}$: 1 if plant $i$ is open, 0 otherwise |
- $q_{s,i,r} \ge 0$: Quantity of raw material $r$ shipped from supplier $s$ to plant $i$ |
- $x_{i,j,p} \ge 0$: Quantity of product $p$ shipped from plant $i$ to customer $j$ |

---

### **Objective Function**

$$
\min Z =
\underbrace{\sum_i F_i y_i}_{\text{Plant fixed cost}}
+ \underbrace{\sum_{s,i,r} c^R_{s,i,r} q_{s,i,r}}_{\text{Raw transport cost}}
+ \underbrace{\sum_{i,j,p} c^P_{i,j,p} x_{i,j,p}}_{\text{Product transport cost}}
$$

---

### **Constraints**

#### (1) Customer Demand Satisfaction
$$
\sum_i x_{i,j,p} = d_{j,p} \quad \forall j,p
$$

#### (2) Supplier Capacity
$$
\sum_i q_{s,i,r} \le \text{Cap}^R_{s,r} \quad \forall s,r
$$

#### (3) Plant Production Capacity
$$
\sum_{j,p} x_{i,j,p} \le \text{Cap}^P_i \, y_i \quad \forall i
$$

#### (4) BOM Conversion (Material Balance)
$$
\sum_s q_{s,i,r} = \sum_p \alpha_{i,r,p} \sum_j x_{i,j,p} \quad \forall i,r
$$

#### (5) Variable Domains
$$
q_{s,i,r} \ge 0, \quad x_{i,j,p} \ge 0, \quad y_i \in \{0,1\}
$$

---

**Model Highlights**

- Each **plant** transforms multiple raw materials into one or more finished products via $\alpha_{i,r,p}$.  
- **Suppliers** provide raw materials under capacity constraints.  
- **Customers** have specific product demands.  
- **Plants** decide whether to open and how much to produce and ship.  
- The **objective** minimizes total cost including plant setup, raw material transport, and product transport.

---

### **Possible Extensions**

- Add **time-indexed** capacity expansions (multi-period).  
- Add **emission limits** or **carbon costs**.  
- Add **tiered suppliers** or **shared raw materials** across multiple products.  


In [39]:
from pyomo.environ import *

def multi_product_bom_model(solver_name='gurobi'):

    # --- Sets ---
    S = ['Supp1', 'Supp2']
    I = ['PlantA', 'PlantB']
    J = ['Cust1', 'Cust2']
    R = ['Steel', 'Plastic']
    P = ['Widget', 'Gadget']

    # --- Parameters ---
    demand = {
        ('Cust1', 'Widget'): 40, ('Cust1', 'Gadget'): 30,
        ('Cust2', 'Widget'): 60, ('Cust2', 'Gadget'): 40
    }
    capR = {('Supp1', 'Steel'): 150, ('Supp1', 'Plastic'): 100,
            ('Supp2', 'Steel'): 120, ('Supp2', 'Plastic'): 80}
    capP = {'PlantA': 200, 'PlantB': 180}
    F = {'PlantA': 15000, 'PlantB': 12000}

    # Transportation costs
    cR = {}
    for s in S:
        for i in I:
            for r in R:
                cR[(s, i, r)] = 3 + (0.5 if s == 'Supp2' else 0)

    cP = {}
    for i in I:
        for j in J:
            for p in P:
                cP[(i, j, p)] = 6 if i == 'PlantA' else 5.5

    # BOM conversion coefficients
    # e.g., each product uses some mix of steel/plastic
    alpha = {}
    for i in I:
        for r in R:
            for p in P:
                alpha[(i, r, p)] = 0.5 if r == 'Steel' else 0.3

    # --- Model ---
    model = ConcreteModel()
    model.S = Set(initialize=S)
    model.I = Set(initialize=I)
    model.J = Set(initialize=J)
    model.R = Set(initialize=R)
    model.P = Set(initialize=P)

    model.d = Param(model.J, model.P, initialize=lambda m,j,p: demand.get((j,p),0))
    model.capR = Param(model.S, model.R, initialize=lambda m,s,r: capR.get((s,r),0))
    model.capP = Param(model.I, initialize=capP)
    model.F = Param(model.I, initialize=F)
    model.cR = Param(model.S, model.I, model.R, initialize=lambda m,s,i,r: cR.get((s,i,r),1e6))
    model.cP = Param(model.I, model.J, model.P, initialize=lambda m,i,j,p: cP.get((i,j,p),1e6))
    model.alpha = Param(model.I, model.R, model.P, initialize=lambda m,i,r,p: alpha.get((i,r,p),0))

    # --- Variables ---
    model.y = Var(model.I, domain=Binary)
    model.q = Var(model.S, model.I, model.R, domain=NonNegativeReals)
    model.x = Var(model.I, model.J, model.P, domain=NonNegativeReals)

    # --- Objective ---
    def obj_rule(m):
        return (sum(m.F[i] * m.y[i] for i in m.I)
              + sum(m.cR[s, i, r] * m.q[s, i, r] for s in m.S for i in m.I for r in m.R)
              + sum(m.cP[i, j, p] * m.x[i, j, p] for i in m.I for j in m.J for p in m.P))
    model.OBJ = Objective(rule=obj_rule, sense=minimize)

    # --- Constraints ---
    def demand_rule(m, j, p):
        return sum(m.x[i, j, p] for i in m.I) == m.d[j, p]
    model.Demand = Constraint(model.J, model.P, rule=demand_rule)

    def supplier_cap_rule(m, s, r):
        return sum(m.q[s, i, r] for i in m.I) <= m.capR[s, r]
    model.SuppCap = Constraint(model.S, model.R, rule=supplier_cap_rule)

    def plant_cap_rule(m, i):
        return sum(m.x[i, j, p] for j in m.J for p in m.P) <= m.capP[i] * m.y[i]
    model.PlantCap = Constraint(model.I, rule=plant_cap_rule)

    def bom_balance_rule(m, i, r):
        return sum(m.q[s, i, r] for s in m.S) == sum(m.alpha[i, r, p] * sum(m.x[i, j, p] for j in m.J) for p in m.P)
    model.BOM = Constraint(model.I, model.R, rule=bom_balance_rule)

    # --- Solve ---
    solver = SolverFactory(solver_name)
    solver.solve(model, tee=True)

    print(f"\nOptimal Objective: {value(model.OBJ):,.2f}")
    print("\nOpen Plants:")
    for i in I:
        print(f"  {i}: {int(round(value(model.y[i])))}")

    print("\nProduct Shipments:")
    for i in I:
        for j in J:
            for p in P:
                if value(model.x[i,j,p]) > 1e-6:
                    print(f"  {i} -> {j} ({p}): {value(model.x[i,j,p]):.1f}")

    print("\nRaw Material Shipments:")
    for s in S:
        for i in I:
            for r in R:
                if value(model.q[s,i,r]) > 1e-6:
                    print(f"  {s} -> {i} ({r}): {value(model.q[s,i,r]):.1f}")

multi_product_bom_model('gurobi')


Read LP format model from file /tmp/tmpkcjmyfoo.pyomo.lp
Reading time = 0.00 seconds
x1: 14 rows, 18 columns, 50 nonzeros
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 24.04.1 LTS")

CPU model: 12th Gen Intel(R) Core(TM) i7-12700H, instruction set [SSE2|AVX|AVX2]
Thread count: 20 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 14 rows, 18 columns and 50 nonzeros
Model fingerprint: 0xbf12cc61
Variable types: 16 continuous, 2 integer (2 binary)
Coefficient statistics:
  Matrix range     [3e-01, 2e+02]
  Objective range  [3e+00, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+01, 2e+02]
Presolve removed 4 rows and 7 columns
Presolve time: 0.00s
Presolved: 10 rows, 11 columns, 28 nonzeros
Variable types: 9 continuous, 2 integer (2 binary)
Found heuristic solution: objective 28343.000000

Root relaxation: objective 1.334300e+04, 3 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Obj

In the solution above, Plant A is closed and Plant B is open. Supplier 1 supplies 85.0 units of Steel and 51.0 units of plastic to Plant B, which in turn supplies 40.0 units and 30.0 units, respectively of Widgets and Gadgets to Customer 1 and 60.0 units and 40.0 units of Gadget of Customer 2.

## 5) Inventory, lead time & service level (mathematical explanation)

For a warehouse with lead time $L$ (periods) and demand std $\sigma$ per period, approximate safety stock for cycle service level $\alpha$ by
$$SS = z_{\alpha}\,\sigma\,\sqrt{L}$$
where $z_{\alpha}$ is the normal quantile (e.g., $z_{0.95}\approx 1.645$).

We include safety stock as a lower bound on inventory when a facility is opened and charge holding cost for safety stock.



**Sets & Indices**:
- $I$: warehouses, $J$: customers.

**Parameters**:
- $L_i$: lead time for warehouse $i$.
- $\sigma$: per-period demand standard deviation (or per-customer aggregated std).
- $\alpha$: desired cycle service level (e.g., 0.95).

**Decision variables**:
- $y_i$: binary open decision.
- $x_{i,j}$: shipments.
- $inv_i$: safety stock held at warehouse $i$.

**Safety stock**: precompute $SS_i = z_\alpha \sigma \sqrt{L_i}$ (where $z_\alpha$ is the normal quantile) and enforce $inv_i \ge SS_i y_i$.


In [34]:
# Inventory + lead time + safety-stock Pyomo example
from pyomo.environ import *
import math

def inventory_pyomo(solver_name='gurobi'):
    I = ['WH1','WH2']
    J = ['C1','C2','C3']
    demand = {'C1':30, 'C2':50, 'C3':20}
    leadtime = {'WH1':2, 'WH2':5}
    daily_std = 5.0
    z95 = 1.645
    safety = {i: z95 * daily_std * math.sqrt(leadtime[i]) for i in I}

    holding = {'WH1':0.5, 'WH2':0.4}
    open_cost = {'WH1':5000, 'WH2':4000}
    cap = {'WH1':150, 'WH2':120}

    trans = {('WH1','C1'):4, ('WH1','C2'):6, ('WH1','C3'):5,
             ('WH2','C1'):5, ('WH2','C2'):4, ('WH2','C3'):6}

    model = ConcreteModel()
    model.I = Set(initialize=I)
    model.J = Set(initialize=J)
    model.d = Param(model.J, initialize=demand)
    model.ss = Param(model.I, initialize=safety)
    model.h = Param(model.I, initialize=holding)
    model.f = Param(model.I, initialize=open_cost)
    model.u = Param(model.I, initialize=cap)
    model.t = Param(model.I, model.J, initialize=lambda m,i,j: trans[(i,j)])

    model.y = Var(model.I, domain=Binary)
    model.x = Var(model.I, model.J, domain=NonNegativeReals)
    model.inv = Var(model.I, domain=NonNegativeReals)  # safety stock held

    def obj_rule(m):
        return sum(m.f[i]*m.y[i] for i in m.I) + sum(m.t[i,j]*m.x[i,j] for i in m.I for j in m.J) + sum(m.h[i]*m.inv[i] for i in m.I)
    model.OBJ = Objective(rule=obj_rule, sense=minimize)

    def demand_rule(m,j):
        return sum(m.x[i,j] for i in m.I) == m.d[j]
    model.Demand = Constraint(model.J, rule=demand_rule)

    def cap_rule(m,i):
        return sum(m.x[i,j] for j in m.J) + m.inv[i] <= m.u[i]*m.y[i]
    model.Cap = Constraint(model.I, rule=cap_rule)

    def ss_rule(m,i):
        return m.inv[i] >= m.ss[i]*m.y[i]
    model.SS = Constraint(model.I, rule=ss_rule)

    solver = SolverFactory(solver_name)
    solver.solve(model, tee=True)
    print('Objective:', value(model.OBJ))
    for i in I:
        print(i, 'open=', int(round(value(model.y[i]))), 'safety_stock=', value(model.inv[i]))
    for i in I:
        for j in J:
            q = value(model.x[i,j])
            if q > 1e-6:
                print('Ship', q, 'from', i, 'to', j)

inventory_pyomo('gurobi')


Read LP format model from file /tmp/tmpy3itwbf1.pyomo.lp
Reading time = 0.00 seconds
x1: 7 rows, 10 columns, 20 nonzeros
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 24.04.1 LTS")

CPU model: 12th Gen Intel(R) Core(TM) i7-12700H, instruction set [SSE2|AVX|AVX2]
Thread count: 20 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 7 rows, 10 columns and 20 nonzeros
Model fingerprint: 0x585f68a4
Variable types: 8 continuous, 2 integer (2 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [4e-01, 5e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 5e+01]
Presolve removed 7 rows and 10 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 20 available processors)

Solution count 1: 4477.36 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.477356663646e+03, best bo

The solution above says that only Warehouse 2 should be open and should have a safety stock of 18.39 units. Also, 30.0 units are to be shipped from Warehouse 2 (WH2) to customer C1, 50.0 units from Warehouse 2 to customer C2 and 20.0 units from Warehouse 2 to Customer C3.

## 6) Benders Decomposition for Facility Location Problem 

We consider a **capacitated facility location problem** where we must decide:
- Which facilities to open (binary decision variables),
- How to transport customer demand from open facilities (continuous flow variables).

**Sets and Indices**
- $I$: Set of candidate facility locations (e.g., factories or warehouses).  
- $J$: Set of customer zones (demand points).

**Parameters**
- $f_i$: Fixed cost of opening facility $i \in I$.  
- $u_i$: Capacity of facility $i$.  
- $d_j$: Demand of customer $j \in J$.  
- $c_{ij}$: Unit transportation cost from facility $i$ to customer $j$.

**Decision Variables**
- $y_i \in \{0,1\}$: 1 if facility $i$ is opened, 0 otherwise.  
- $x_{ij} \ge 0$: Quantity transported from facility $i$ to customer $j$.

**Original MILP Formulation**

The full mixed-integer linear programming (MILP) model is:

$$
\begin{aligned}
\min_{x, y} \quad & \sum_{i \in I} f_i y_i + \sum_{i \in I} \sum_{j \in J} c_{ij} x_{ij} \\
\text{s.t.} \quad 
& \sum_{i \in I} x_{ij} = d_j, && \forall j \in J \quad \text{(demand satisfaction)} \\
& \sum_{j \in J} x_{ij} \le u_i y_i, && \forall i \in I \quad \text{(capacity limits)} \\
& y_i \in \{0,1\}, && \forall i \in I \\
& x_{ij} \ge 0, && \forall i \in I, \; j \in J
\end{aligned}
$$

**Decomposition Strategy**

Benders decomposition separates the **binary decisions** ($y_i$) and the **continuous flow decisions** ($x_{ij}$).

**Master Problem (MP)**
The master controls facility openings ($y_i$) and includes a continuous variable $\theta$ to represent the transportation cost from the subproblem.

$$
\begin{aligned}
\min_{y, \theta} \quad & \sum_{i \in I} f_i y_i + \theta \\
\text{s.t.} \quad 
& \sum_{i \in I} u_i y_i \ge \sum_{j \in J} d_j, \\
& \theta \ge \text{Benders Cuts (added iteratively)}, \\
& y_i \in \{0,1\}, \quad \theta \ge 0.
\end{aligned}
$$

**Subproblem (SP)**
Given a fixed $y^t$ from the master, the subproblem determines the best flows $x_{ij}$:

$$
\begin{aligned}
\min_{x} \quad & \sum_{i \in I} \sum_{j \in J} c_{ij} x_{ij} \\
\text{s.t.} \quad 
& \sum_{i \in I} x_{ij} = d_j, && \forall j \in J \\
& \sum_{j \in J} x_{ij} \le u_i y_i^t, && \forall i \in I \\
& x_{ij} \ge 0.
\end{aligned}
$$

**Dual of the Subproblem**

Let:
- $\pi_j$: dual variable for demand constraint $\sum_i x_{ij} = d_j$.  
- $\alpha_i$: dual variable for capacity constraint $\sum_j x_{ij} \le u_i y_i^t$.

Then, the **dual problem** is:

$$
\begin{aligned}
\max_{\pi, \alpha} \quad & \sum_{j \in J} \pi_j d_j - \sum_{i \in I} \alpha_i u_i y_i^t \\
\text{s.t.} \quad 
& \pi_j - \alpha_i \le c_{ij}, && \forall i \in I, \; j \in J \\
& \alpha_i \ge 0, \quad \forall i \in I.
\end{aligned}
$$

The dual provides information to generate **Benders cuts**.

**Benders Optimality Cut**

Given dual solution $(\pi^t, \alpha^t)$ for a fixed $y^t$,  
we can construct an *optimality cut*:

$$
\theta \ge \sum_{j \in J} \pi_j^t d_j - \sum_{i \in I} \alpha_i^t u_i y_i
$$

This inequality ensures that $\theta$ (the estimated subproblem cost) is at least as large as the true transportation cost.

**Feasibility Cut**

The feasibitity cut is generated when the subproblem is infeasible. By Farkas's lemma, there exist multipliers $(\phi_j)_{j \in J}$ and $(\gamma_i)_{i \in I}$ such that 

$$
\sum_{j \in J} \phi_j d_j + \sum_{i \in I} \gamma_i (u_i y_i^t) < 0
$$

The feasiblity cut that is added to the master problem then takes the form:

$$
\sum_{i \in I} \gamma_i (u_i y_i) \geq - \sum_{j \in J} \phi_j d_j
$$

This serves to exclude the solution $y^t$ since it violates the above constraint.

**Algorithm Summary**

1. **Initialize** the master problem with no Benders cuts.  
2. **Solve the master** to get $y^t$ and $\theta^t$.  
3. **Solve the subproblem** (LP) for the fixed $y^t$:
   - If **feasible**, generate an optimality cut.  
   - If **infeasible**, generate a feasibility cut.  
4. **Add the cut** to the master.  
5. Repeat until convergence (no violated cuts).

---

**Lazy Constraint Implementation in Gurobi**

Instead of manually looping, Gurobi’s **lazy constraint callback** allows:
- Adding Benders cuts dynamically during branch-and-bound.
- Avoiding repeated solves of the master problem.

Each time Gurobi finds a new integer solution for $y_i$, the callback:
1. Solves the subproblem LP for that $y^t$.
2. Obtains $(\pi^t, \alpha^t)$ duals.
3. Adds a cut:

$$
\theta \ge \sum_{j} \pi_j^t d_j - \sum_i \alpha_i^t u_i y_i
$$

using the command:

```python
model.cbLazy(theta >= const_term + quicksum(coeff[i]*y[i] for i in I))



In [19]:
"""
benders_with_farkas_full.py

Callback-based Benders decomposition with feasibility cuts derived from the Farkas dual ray.
Uses gurobipy (Gurobi direct API).
"""

import random, math
from gurobipy import Model, GRB, quicksum

# -------------------------
# Data generator (robust to ensure total capacity >= total demand)
# -------------------------
def generate_instance(n_fac=6, n_cust=20, seed=1, capacity_scale=1.2):
    random.seed(seed)
    I = [f"F{i+1}" for i in range(n_fac)]
    J = [f"C{j+1}" for j in range(n_cust)]
    # demands
    d = {j: random.randint(40, 100) for j in J}
    total_d = sum(d.values())
    # capacities: ensure aggregate capacity >= total_d
    avg_needed = math.ceil(total_d / n_fac)
    # build capacities around avg_needed * capacity_scale
    u = {i: max(1, int(avg_needed * capacity_scale + random.randint(-30, 50))) for i in I}
    # fixed costs and transport costs
    f = {i: random.randint(4000, 8000) for i in I}
    c = {(i, j): round(1.0 + random.random() * 3.5, 2) for i in I for j in J}
    return {"I": I, "J": J, "d": d, "u": u, "f": f, "c": c, "total_d": total_d}

# -------------------------
# Persistent subproblem (transport LP)
# -------------------------
def build_persistent_subproblem(data):
    I, J, c, d, u = data["I"], data["J"], data["c"], data["d"], data["u"]
    sub = Model("sub_lp")
    sub.Params.OutputFlag = 0
    # decision variables x[i,j]
    x = sub.addVars(I, J, lb=0.0, name="x")
    sub.update()
    # demand equalities: sum_i x_ij == d_j
    demand_con = sub.addConstrs((quicksum(x[i, j] for i in I) == d[j] for j in J), name="Demand")
    # capacity inequalities (RHS will be set to u[i]*y_i in callback)
    cap_con = sub.addConstrs((quicksum(x[i, j] for j in J) <= u[i] for i in I), name="Cap")
    # objective: minimize transport
    sub.setObjective(quicksum(c[i, j] * x[i, j] for i in I for j in J), GRB.MINIMIZE)
    sub.update()
    # store references
    sub._x = x
    sub._demand_con = demand_con
    sub._cap_con = cap_con
    sub._I = I
    sub._J = J
    return sub

# -------------------------
# Master model (facility selection + theta)
# -------------------------
def build_master(data, enable_agg_cut=True):
    I = data["I"]
    f = data["f"]
    total_d = data["total_d"]
    u = data["u"]

    master = Model("master")
    master.Params.OutputFlag = 0
    # allow lazy constraints in callbacks
    master.Params.LazyConstraints = 1

    y = master.addVars(I, vtype=GRB.BINARY, name="y")
    theta = master.addVar(lb=0.0, name="theta")

    master.setObjective(quicksum(f[i] * y[i] for i in I) + theta, GRB.MINIMIZE)

    # optional aggregate capacity cut to avoid trivial infeasibility
    if enable_agg_cut:
        master.addConstr(quicksum(u[i] * y[i] for i in I) >= total_d, name="agg_capacity")

    # store handles for callback
    master._y = y
    master._theta = theta
    master._data = data
    return master

# -------------------------
# Callback with optimality + Farkas feasibility handling
# -------------------------
def make_benders_callback(master, sub, tol=1e-8, verbose=False):
    y_vars = master._y
    theta_var = master._theta
    data = master._data
    I, J, u, d = data["I"], data["J"], data["u"], data["d"]
    total_d = data["total_d"]

    def callback(model, where):
        # act on integer feasible solutions only
        if where == GRB.Callback.MIPSOL:
            # read candidate y and theta
            y_cand = {i: int(round(model.cbGetSolution(y_vars[i]))) for i in I}
            theta_cand = model.cbGetSolution(theta_var)
            # quick aggregate check
            cap_sum = sum(u[i] * y_cand[i] for i in I)
            if cap_sum < total_d - 1e-9:
                # add aggregate capacity feasibility cut
                if verbose:
                    print("[Callback] Aggregate capacity violated by candidate y; adding agg cut.")
                model.cbLazy(quicksum(u[i] * y_vars[i] for i in I) >= total_d)
                return

            # update subproblem capacity RHSs to u_i * y_i (persistent subproblem)
            for i in I:
                # set RHS value directly on constraint object
                sub._cap_con[i].RHS = u[i] * y_cand[i]

            # solve the subproblem LP
            sub.optimize()

            # handle statuses
            if sub.Status == GRB.OPTIMAL:
                sub_obj = sub.ObjVal
                # get duals for demand eq (pi) and cap <= (alpha)
                pi = {j: sub._demand_con[j].Pi for j in J}
                alpha = {i: sub._cap_con[i].Pi for i in I}

                rhs_const = sum(pi[j] * d[j] for j in J)
                coeffs = {i: -u[i] * alpha[i] for i in I}

                # compute cut RHS at candidate to test violation
                rhs_at_cand = rhs_const + sum(coeffs[i] * y_cand[i] for i in I)
                if theta_cand + tol < rhs_at_cand - 1e-9:
                    # add optimality cut as lazy constraint: theta >= rhs_const + sum coeffs * y
                    expr_rhs = rhs_const + quicksum(coeffs[i] * y_vars[i] for i in I)
                    if verbose:
                        print("[Callback] Adding optimality cut. rhs_const={:.6f}, coeffs_sample={}".format(rhs_const, dict(list(coeffs.items())[:3])))
                    model.cbLazy(theta_var >= expr_rhs)
                else:
                    if verbose:
                        print("[Callback] Candidate satisfies optimality cut (no cut added).")
                return

            # If infeasible: attempt to extract Farkas dual ray and construct feasibility cut
            if sub.Status == GRB.INFEASIBLE:
                if verbose:
                    print("[Callback] Subproblem INFEASIBLE for y=", y_cand)
                # Try to get Farkas duals from Gurobi constraints
                try:
                    phi = {j: sub._demand_con[j].FarkasDual for j in J}
                    gamma = {i: sub._cap_con[i].FarkasDual for i in I}
                except Exception as e:
                    # Fallback: if FarkasDual not available, add aggregate capacity cut
                    if verbose:
                        print("[Callback] FarkasDual not available or error:", str(e))
                        print("[Callback] Adding fallback aggregate capacity cut.")
                    model.cbLazy(quicksum(u[i] * y_vars[i] for i in I) >= total_d)
                    return

                # Build feasibility cut from Farkas ray (sign convention careful)
                # Following algebra (see explanation in text):
                #   phi_j correspond to equality multipliers; gamma_i correspond to <= multipliers
                # Feasibility certificate roughly: sum_j phi_j * d_j + sum_i gamma_i * (u_i * y_i) > 0
                # Translate to cut: sum_i (gamma_i * u_i) * y_i >= - sum_j phi_j * d_j
                coeffs_y = {i: gamma[i] * u[i] for i in I}
                rhs_const = - sum(phi[j] * d[j] for j in J)

                # Evaluate at candidate to verify violation (should be violated)
                lhs_at_cand = sum(coeffs_y[i] * y_cand[i] for i in I)
                if verbose:
                    print("[Callback] Farkas (phi sample, gamma sample):", list(phi.items())[:3], list(gamma.items())[:3])
                    print("[Callback] Feasibility cut prototype: sum(coeffs_y * y) >= {:.6f}".format(rhs_const))
                    print("[Callback] LHS at candidate:", lhs_at_cand, "RHS:", rhs_const)

                # If the inequality is violated (lhs < rhs), add it. If not, flip signs try (robustness).
                violated = lhs_at_cand + 1e-9 < rhs_const - 1e-9
                if not violated:
                    # try sign flips: sometimes Gurobi returns opposite sign conventions
                    # try using -phi and -gamma
                    coeffs_y_alt = {i: -gamma[i] * u[i] for i in I}
                    rhs_const_alt = - sum((-phi[j]) * d[j] for j in J)
                    lhs_alt = sum(coeffs_y_alt[i] * y_cand[i] for i in I)
                    if verbose:
                        print("[Callback] Trying flipped-sign feasibility cut attempt.")
                        print("[Callback] LHS_alt:", lhs_alt, "RHS_alt:", rhs_const_alt)
                    if lhs_alt + 1e-9 < rhs_const_alt - 1e-9:
                        coeffs_y = coeffs_y_alt
                        rhs_const = rhs_const_alt
                        violated = True
                # If still not violated, fallback to aggregate capacity cut
                if not violated:
                    if verbose:
                        print("[Callback] Farkas-derived cut not violated at candidate (unexpected). Adding aggregate capacity fallback.")
                    model.cbLazy(quicksum(u[i] * y_vars[i] for i in I) >= total_d)
                    return

                # Add the feasibility cut (lazy): quicksum(coeffs_y[i] * y_i) >= rhs_const
                expr = quicksum(coeffs_y[i] * y_vars[i] for i in I)
                if verbose:
                    print("[Callback] Adding feasibility (Farkas) cut.")
                model.cbLazy(expr >= rhs_const)
                return

            # other statuses: fallback
            if verbose:
                print("[Callback] Subproblem status not optimal/infeasible; adding aggregate capacity fallback.")
            model.cbLazy(quicksum(u[i] * y_vars[i] for i in I) >= total_d)

    return callback

# -------------------------
# Runner
# -------------------------
def run_benders(n_fac=6, n_cust=20, seed=123, verbose=False):
    # prepare data
    data = generate_instance(n_fac=n_fac, n_cust=n_cust, seed=seed, capacity_scale=1.2)
    # quick feasibility check
    if sum(data["u"].values()) < data["total_d"]:
        raise RuntimeError("Generated instance infeasible: total capacity < total demand")

    # build persistent subproblem and master
    sub = build_persistent_subproblem(data)
    master = build_master(data, enable_agg_cut=True)

    # create callback
    cb = make_benders_callback(master, sub, tol=1e-8, verbose=verbose)

    # solve master with callback
    if verbose:
        master.Params.OutputFlag = 1
    else:
        master.Params.OutputFlag = 0
    master.optimize(cb)

    # report
    if master.Status == GRB.OPTIMAL or master.Status == GRB.TIME_LIMIT:
        y_sol = {i: int(master._y[i].X) for i in data["I"]}
        theta_sol = master._theta.X
        print("\n*** SOLUTION SUMMARY ***")
        print("Facilities open:", [i for i in data["I"] if y_sol[i] == 1])
        print("Theta (transport cost):", theta_sol)
        print("Master objective (total cost):", master.ObjVal)
    else:
        print("Master terminated with status:", master.Status)
        if master.Status == GRB.INFEASIBLE:
            print("Master infeasible: instance likely has insufficient total capacity.")

# -------------------------
# Main
# -------------------------
if __name__ == "__main__":
    # small instance for quick run; increase sizes for stress tests
    run_benders(n_fac=6, n_cust=20, seed=42, verbose=True)



Set parameter OutputFlag to value 1
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 24.04.1 LTS")

CPU model: 12th Gen Intel(R) Core(TM) i7-12700H, instruction set [SSE2|AVX|AVX2]
Thread count: 20 physical cores, 20 logical processors, using up to 20 threads

Non-default parameters:
LazyConstraints  1

Optimize a model with 1 rows, 7 columns and 6 nonzeros
Model fingerprint: 0x9c0cc673
Variable types: 1 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [2e+02, 3e+02]
  Objective range  [1e+00, 7e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+03, 1e+03]
[Callback] Adding optimality cut. rhs_const=2077.660000, coeffs_sample={'F1': 206.66999999999996, 'F2': 50.200000000000045, 'F3': -0.0}
[Callback] Adding optimality cut. rhs_const=2077.660000, coeffs_sample={'F1': 206.66999999999996, 'F2': 50.200000000000045, 'F3': -0.0}
[Callback] Adding optimality cut. rhs_const=2098.530000, coeffs_sample={'F1': -0.0, 'F2': 97.88999999999997,

In the above solution, the Benders decomposition converged in 3 optimality cut insertions and 1 branch-and-bound node. The solution says that facilities F1, F3, F4, F5 and F6 are to remain open and the total transportation costs to send the goods to the 20 customers from these facilities is 2417.13 and total cost is 32714.13.

## 7) Sample CSVs and utilities

Typically the data for these models will in the form of CSVs. So as an example, we created small sample CSVs in `supply_chain_csvs` and demonstrate loading them.

In [23]:
# CSV loading utilities
import pandas as pd
def load_sample_inputs(data_dir='supply_chain_csvs'):
    demand = pd.read_csv(f'{data_dir}/customers.csv')
    facilities = pd.read_csv(f'{data_dir}/facilities.csv')
    suppliers = pd.read_csv(f'{data_dir}/suppliers.csv')
    print('Loaded:', len(demand),'demand rows,', len(facilities),'facilities,', len(suppliers),'suppliers')
    return demand, facilities, suppliers

load_sample_inputs('supply_chain_csvs')


Loaded: 3 demand rows, 3 facilities, 2 suppliers


(  CustomerID  Demand_ProductA  Demand_ProductB Location
 0         C1              100               50    North
 1         C2               80               40     East
 2         C3               60               30     West,
   FacilityID  Capacity_ProductA  Capacity_ProductB  FixedCost Location
 0         F1                120                 70      20000    North
 1         F2                100                 50      18000     East
 2         F3                 80                 40      15000     West,
   SupplierID  SupplyCap_ProductA  SupplyCap_ProductB  FixedCost Location
 0         S1                 150                 100      10000    North
 1         S2                 200                 150      12000    South)

## 7) Mock AWS integration & real `boto3` snippets

This section contains:
- Mock classes for S3, Parameter Store, and Batch to demonstrate notebook integration without credentials.
- Real `boto3` snippet examples (do **not** include credentials in the notebook).

Use IAM roles (Batch compute env / EC2 / EKS) to grant access to S3 and SSM in production.


In [25]:
# Mock AWS integration — demo and real boto3 snippets
import json

class MockS3:
    def __init__(self):
        self.storage = {}
    def upload(self, bucket, key, data):
        print(f"[MockS3] Upload s3://{bucket}/{key}")
        self.storage[(bucket,key)] = data
    def download(self, bucket, key):
        print(f"[MockS3] Download s3://{bucket}/{key}")
        return self.storage.get((bucket,key), None)

class MockParamStore:
    def __init__(self):
        self.params = {'solver':'gurobi','MIPGap':'0.01','TimeLimit':'600'}
    def get(self, name):
        print(f"[MockParamStore] get {name}")
        return self.params.get(name, None)

class MockBatch:
    def submit_job(self, job_name, job_queue, job_definition, parameters):
        print(f"[MockBatch] submit {job_name} to {job_queue} def {job_definition}")
        print("params:", json.dumps(parameters, indent=2))
        return {'jobId':'mock-0001'}

# Demo usage
s3 = MockS3()
ps = MockParamStore()
batch = MockBatch()

s3.upload('scm-bucket','inputs/demand.csv','C1,80\nC2,70\nC3,60')
print(s3.download('scm-bucket','inputs/demand.csv'))
print('Param solver:', ps.get('solver'))

job = batch.submit_job('greenfield','scm-queue','pyomo-solver',{'input':'s3://scm-bucket/inputs/demand.csv'})
print('Submitted job:', job)
print('')


# Real boto3 snippet (example only; DO NOT run here with credentials)
real_snippet = '''
import boto3
s3 = boto3.client('s3')
s3.upload_file('demand.csv','my-bucket','inputs/demand.csv')

ssm = boto3.client('ssm')
resp = ssm.get_parameter(Name='SCM_solver_config', WithDecryption=True)
config = json.loads(resp['Parameter']['Value'])

batch = boto3.client('batch')
batch.submit_job(jobName='greenfield-run', jobQueue='my-queue', jobDefinition='pyomo-solver:1', containerOverrides={'environment':[{'name':'INPUT_S3','value':'s3://my-bucket/inputs/demand.csv'}]})
'''
print("Real boto3 snippet (example):\n", real_snippet)


[MockS3] Upload s3://scm-bucket/inputs/demand.csv
[MockS3] Download s3://scm-bucket/inputs/demand.csv
C1,80
C2,70
C3,60
[MockParamStore] get solver
Param solver: gurobi
[MockBatch] submit greenfield to scm-queue def pyomo-solver
params: {
  "input": "s3://scm-bucket/inputs/demand.csv"
}
Submitted job: {'jobId': 'mock-0001'}

Real boto3 snippet (example):
 
import boto3
s3 = boto3.client('s3')
s3.upload_file('demand.csv','my-bucket','inputs/demand.csv')

ssm = boto3.client('ssm')
resp = ssm.get_parameter(Name='SCM_solver_config', WithDecryption=True)
config = json.loads(resp['Parameter']['Value'])

batch = boto3.client('batch')
batch.submit_job(jobName='greenfield-run', jobQueue='my-queue', jobDefinition='pyomo-solver:1', containerOverrides={'environment':[{'name':'INPUT_S3','value':'s3://my-bucket/inputs/demand.csv'}]})



## Final notes

This comprehensive notebook contains basic → advanced models, Benders implementation (Pyomo+Gurobi iterative), and mock AWS integration.  